In [1]:
import os
from dotenv import load_dotenv
from data_extraction import DataExtractor
from database_utils import DatabaseConnector
from data_cleaning import DataCleaning


Creating instance of the DatabaseConnector

Initialising the db engine and listing the tables: 
- 'legacy_store_details' 
- 'legacy_users'
- 'orders_table'

Then creating instances of extractor and data cleaning object

In [2]:
RDS_CONNECTOR = DatabaseConnector()
RDS_CONNECTOR.init_db_engine()
RDS_CONNECTOR.list_db_tables()

lamb = DataExtractor()
test_clean = DataCleaning()

['legacy_store_details', 'legacy_users', 'orders_table']


Use read_rds_table to extract the legacy users table

Cleaning and uploading user data

In [3]:
# ['legacy_store_details', 'legacy_users', 'orders_table']
# print(lamb.read_rds_table('legacy_users'))

## User Data
retrieved_user = lamb.read_rds_table('legacy_users')
cleaned_user = test_clean.clean_user_data(retrieved_user)
RDS_CONNECTOR.upload_to_db(cleaned_user, 'dim_users')

15284


InternalError: (psycopg2.errors.DependentObjectsStillExist) cannot drop table dim_users because other objects depend on it
DETAIL:  constraint fk_user_uuid on table orders_table depends on table dim_users
HINT:  Use DROP ... CASCADE to drop the dependent objects too.

[SQL: 
DROP TABLE dim_users]
(Background on this error at: https://sqlalche.me/e/20/2j85)

Extracted from pdf file using tabula

Cleaning and uploading card data

In [4]:
## Card Data
retrieved_data = lamb.retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')
#print(retrieved_data)
cleaned_card_data = test_clean.clean_card_data(retrieved_data)
RDS_CONNECTOR.upload_to_db(cleaned_card_data, 'dim_card_details')

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


15284
         card_number expiry_date                card_provider  \
0     30060773296197       09/26  Diners Club / Carte Blanche   
1    349624180933183       10/23             American Express   
2   3529023891650490       06/23                 JCB 16 digit   
3    213142929492281       09/27                 JCB 15 digit   
4       502067329974       10/25                      Maestro   
..               ...         ...                          ...   
14   180036921556789       12/28                 JCB 15 digit   
15   180018030448512       11/24                 JCB 15 digit   
16  3569953313547220       04/24                 JCB 16 digit   
17  4444521712606810       06/27                VISA 16 digit   
18   372031786522735       02/30             American Express   

   date_payment_confirmed  
0              2015-11-25  
1              2001-06-18  
2              2000-12-26  
3              2011-02-12  
4              1997-03-13  
..                    ...  
14             19

InternalError: (psycopg2.errors.DependentObjectsStillExist) cannot drop table dim_card_details because other objects depend on it
DETAIL:  constraint fk_card_number on table orders_table depends on table dim_card_details
HINT:  Use DROP ... CASCADE to drop the dependent objects too.

[SQL: 
DROP TABLE dim_card_details]
(Background on this error at: https://sqlalche.me/e/20/2j85)

Find the number of stores which is 451 (retrieved from api)

The extracted the store data 

Cleaning and uploading store data

In [ ]:
## Store data
load_dotenv()
api_key = os.getenv("API_KEY")
endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'

headers = {'x-api-key': api_key}

number_of_stores = lamb.list_number_of_stores(endpoint, headers)

other = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'
stores_data = lamb.retrieve_stores_data(other, headers, number_of_stores)
cleaned_stores_data = test_clean.clean_store_data(stores_data)
RDS_CONNECTOR.upload_to_db(cleaned_stores_data, 'dim_store_details')

Extract the the products csv from the s3 bucket

Cleaning and uploading product data

In [ ]:
### Product Data
s3_address_products = 's3://data-handling-public/products.csv'
extracted_product_data = lamb.extract_from_s3(s3_address_products)
cleaned_product_data = test_clean.clean_products_data(extracted_product_data)
RDS_CONNECTOR.upload_to_db(cleaned_product_data, "dim_products")

Use read_rds_table to extract the orders_table

Cleaning and uploading orders data

In [5]:
## Order Data
retrieved_orders = lamb.read_rds_table('orders_table')
cleaned_orders = test_clean.clean_orders_data(retrieved_orders)
RDS_CONNECTOR.upload_to_db(cleaned_orders, 'orders_table')

120123
UPLOAD SUCCESSFUL


In [ ]:
print(retrieved_orders.columns)


Extract the time data json file from the s3 bucket

Clean and upload the time data df to db

In [6]:
## Time Data
s3_address_date = 'https://data-handling-public/date_details.json'
extracted_date_data = lamb.extract_from_s3(s3_address_date)
cleaned_date = test_clean.clean_date_data(extracted_date_data)
RDS_CONNECTOR.upload_to_db(cleaned_date, 'dim_date_times')

data-handling-public date_details.json
data-handling-public date_details.json
File has sucessfully retrieved.
Reading the JSON file into DataFrame...
120123
UPLOAD SUCCESSFUL
